## Score a new selfie image and evaluate the acne severity of the face
Given a set of selfie images, this script is used to score the acne severity levels of the selfie face images. We score each skin patch from the selfie and the average score is chosen as the image label. 

The scoring pipelines have the following four steps:
1. Extract skin patches
2. Predict the label for each skin patch
 *  Extract features from CNTK pretrained model
 *  Score the features using the trained full connected neural network model
3. infer the whole face label based on predicted labels of the skin patches from that selfie image. The inferred labels of the entire selfie images in the test directory are output to a csv file. 
4. (Optional) If you also have a csv file with the ground truth labels of the test images, we also compare the predicted label and the ground truth labels, and calculate the RMSE on the golden set. 

***Note***: With the model you trained on Step 3, where random seed = 5 for splitting the training images into training and validation, and initializing the weights of the neural network models, you should get ***RMSE = 0.4819*** on golden set images.

## Prerequisites

### Test images
- Test images should be put in a directory on the machine, and provide the path to this directory to parameter image\_path.
- (***Optional***) Ground truth labels of the test images should be put in a csv file with a headerline, and two columns: Image_Name, Ground_Truth. If this file is None, calculating the RMSE on the test images will be skipped.

### Python, Python libraries and self-defined Python Script
- Python 3.5 or later version 
- CNTK, PIL
- getPatches.py to extract patches from a selfie (modified from ***[Step 1. Extract Forehead, cheeks, and chin skin patches from raw images using facial landmark model and One Eye model](../01_DataPrep/Step 1. Extract Forehead, cheeks, and chin skin patches from raw images using facial landmark model and One Eye model.ipynb)***)
***Note*** You need to mofidy the path to the pretrained landmark model and the Cascade Eye model in the getPatches.py file. 

### Pretrained models
- [Frontal face landmark model](https://github.com/AKSHAYUBHAT/TensorFace/blob/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat) in the same directory as this jupyter notebook.
- [One Eye model](https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml) in the same directory as this jupyter notebook.
- [ResNet152\_ImageNet\_Caffe.model](https://www.cntk.ai/Models/Caffe_Converted/ResNet152_ImageNet_Caffe.model)
- Trained full connected neural network regression model from ***[Step 3. Training_Pipeline](Step3_Training_Pipeline.ipynb)***


## Parameters

In [1]:
pretrained_model_name = 'ResNet152_ImageNet_Caffe.model'
pretrained_model_path = '../models'
pretrained_node_name = 'pool5' 

label_mapping = {1: '1-Clear', 2: '2-Almost Clear', 3: '3-Mild', 4: '4-Moderate', 5: '5-Severe'}

img_path = '../data/labeled_test_images'
test_ground_truth = '../data/test_images_labels.csv' #If None, calculating RMSE on test set will be skipped
result_file = '../data/predicted_labels.csv'
patch_path = '../data/test_images_patches'
regression_model_path = '../models/cntk_regression.dat'
eye_cascade_model = '../models/haarcascade_eye.xml'

image_height = 224 # Here are the image height and width that the skin patches of the testing selfie are going to be resized to.
image_width  = 224 # They have to be the same as the ResNet-152 model requirement.
num_channels = 3

In [2]:
from __future__ import print_function
import os
from os import listdir
from os.path import join, isfile, splitext
import numpy as np
import pandas as pd
import cntk as C
from PIL import Image
import pickle
import time
import json
from cntk import load_model, combine
import cntk.io.transforms as xforms
from cntk.logging import graph
from cntk.logging.graph import get_node_outputs
import getPatches
import cv2

c:\users\deni susanto\appdata\local\programs\python\python36\lib\site-packages\cntk\cntk_py_init.py:84: UserWarning: 

################################################ Missing optional dependency (GPU-Specific) ################################################
   CNTK may crash if the component that depends on those dependencies is loaded.
   Visit https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Windows-Python#optional-gpu-specific-packages for more information.
############################################################################################################################################
If you intend to use CNTK without GPU support, you can ignore the (likely) GPU-specific warning!
############################################################################################################################################

  warnings.warn(WARNING_MSG_GPU_ONLY % ('GPU-Specific', 'https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Windows-Python#optional-gpu-spe

## Extract Skin Patches

In the scoring pipeline, the first step is to extract forehead, left cheek, right cheek, and chin skin patches from the original selfie images. The skin patch images are going to be saved in the directory specified in variable patch\_path.

During this skin patch extraction process, a facial landmark model will be first applied. If face is detected by this model, skin patches will be extracted. If face is not detected by the landmark model, OneEye model will be applied to detect the location of a single eye. Then, forehead and cheek will be extracted based on the eye location.

If neither landmark nor the OneEye model works, the entire selfie will be used to make predictions.

Landmark needs to see both eyes open from the camera. OneEye model works if there is only one open eye identified. Skin patches can be extracted more accurately based on landmark model. So, if possible, we encourage users to face the camera directly with both eyes opened. 

Keep in mind, depending on the angle of the face facing the camera, this step might result in 1, 2, 3, or 4 skin patch images. 


In [3]:
# get the dimension of each patch of images in the testing image directory
dimension_dict = dict()
imageFiles = [f for f in listdir(img_path) if isfile(join(img_path, f))]
for imagefile in imageFiles:
    dim = getPatches.extract_patches(join(img_path, imagefile), {}, {}, patch_path) #extract_patches function is defined in getPatches.py
    dimension_dict[imagefile] = dim

## Predict Image Patch Label

### Load CNTK Pretrained Model

In [4]:
# define pretrained model location, node name
model_file  = os.path.join(pretrained_model_path, pretrained_model_name)
loaded_model  = load_model(model_file)
node_in_graph = loaded_model.find_by_name(pretrained_node_name)
output_nodes  = combine([node_in_graph.owner])

node_outputs = C.logging.get_node_outputs(loaded_model)
for l in node_outputs: 
    if l.name == pretrained_node_name:
        num_nodes = np.prod(np.array(l.shape))
        
print ('the pretrained model is %s' % pretrained_model_name)
print ('the selected layer name is %s and the number of flatten nodes is %d' % (pretrained_node_name, num_nodes))

the pretrained model is ResNet152_ImageNet_Caffe.model
the selected layer name is pool5 and the number of flatten nodes is 2048


In [5]:
def extract_features(image_path):   
    img = Image.open(image_path)       
    resized = img.resize((image_width, image_height), Image.ANTIALIAS)  
    
    bgr_image = np.asarray(resized, dtype=np.float32)[..., [2, 1, 0]]    
    hwc_format = np.ascontiguousarray(np.rollaxis(bgr_image, 2)) 
    
    arguments = {loaded_model.arguments[0]: [hwc_format]}    
    output = output_nodes.eval(arguments)   
    return output

### Load NN model

In [6]:
#load the stored regression model
read_model = pd.read_pickle(regression_model_path)
regression_model = read_model['model'][0]
train_regression = pickle.loads(regression_model)

### Score Image

In [7]:
# get the score value for each patch
patch_score = dict()
for file in next(os.walk(patch_path))[2]:
    file_path = os.path.join(patch_path, file)
    # extract features from CNTK pretrained model
    score_features = extract_features (file_path)[0].flatten()
    # score the extracted features using trained regression model
    pred_score_label = train_regression.predict(score_features.reshape(1,-1))
    patch_score[file] = float("{0:.2f}".format(pred_score_label[0]))

### Check the predicted labels of skin patches

In [8]:
patch_score

{'20171226_203858_landmark_chin.jpg': 1.9,
 '20171226_203858_landmark_fh.jpg': 1.02,
 '20171226_203858_landmark_lc.jpg': 1.01,
 '20171226_203858_landmark_rc.jpg': 1.02,
 '20180727_203313_landmark_chin.jpg': 1.01,
 '20180727_203313_landmark_fh.jpg': 1.04,
 '20180727_203313_landmark_lc.jpg': 2.11,
 '20180727_203313_landmark_rc.jpg': 1.01,
 '20190530_164944_landmark_chin.jpg': 1.01,
 '20190530_164944_landmark_fh.jpg': 1.0,
 '20190530_164944_landmark_lc.jpg': 1.0,
 '20190530_164944_landmark_rc.jpg': 1.0,
 '20200108_084050_landmark_chin.jpg': 2.26,
 '20200108_084050_landmark_fh.jpg': 2.97,
 '20200108_084050_landmark_lc.jpg': 1.6,
 '20200108_084050_landmark_rc.jpg': 2.09,
 'IMG_20170412_185554_landmark_chin.jpg': 1.4,
 'IMG_20170412_185554_landmark_fh.jpg': 2.12,
 'IMG_20170412_185554_landmark_rc.jpg': 2.6,
 'jongun_landmark_chin.jpg': 1.0,
 'jongun_landmark_fh.jpg': 1.02,
 'jongun_landmark_lc.jpg': 1.0,
 'jongun_landmark_rc.jpg': 1.01,
 'me1_landmark_chin.jpg': 1.87,
 'me1_landmark_fh.jpg':

## Infer Image Label

After each skin patch image is predicted, we need to infer the predicted label of the entire selfie from the predicted skin patch labels. In this application, we choose to maximal predicted skin patch label as the predicted label of the entire selfie. This logic can be modified further to balance the performance of the model on the entire 5 levels of the acne severity.

In [9]:
# get the max score value among the patches and record the image name
image_patch_scores = {}
for key in patch_score:
    image_id = key.split("_landmark")[0]
    image_patch_scores_i = image_patch_scores.get(image_id, {"patch_name":[], "patch_score":[]})
    image_patch_scores_i["patch_name"].append(key)
    image_patch_scores_i["patch_score"].append(patch_score[key])
    image_patch_scores[image_id] = image_patch_scores_i

fp = open(result_file, 'w')
fp.write("Image_Name, Predicted_Label_Avg, Most_Severe_Patch\n")

for key in image_patch_scores:
    image_name = key.split("_landmark")[0]
    max_index = np.argmax(image_patch_scores[key]['patch_score'])
    Predicted_Label_Avg = np.mean(image_patch_scores[key]['patch_score'])
    Most_Severe_Patch = image_patch_scores[key]['patch_name'][max_index]
    fp.write('%s, %.4f, %s\n'%(image_name, Predicted_Label_Avg, Most_Severe_Patch))

fp.close()

## (Optional) Calculate RMSE on Test Images

In [13]:
conditions = ["0-Not Acne", "1-Clear", "2-Almost Clear", "3-Mild", "4-Moderate", "5-Severe"]
if test_ground_truth is not None:
    fp = open(test_ground_truth, 'r')
    fp.readline()
    ground_truth = {}
    for row in fp:
        row = row.strip().split(',')
        ground_truth[row[0]] = float(conditions.index(row[1]))
    fp.close()
    
    fp2 = open(result_file, 'r')
    fp2.readline()
    num_images = 0.0
    RMSE = 0.0
    for row in fp2:
        row = row.strip().split(',')
        try:
            RMSE += (ground_truth[row[0].split("_landmark")[0]+".jpg"] - float(row[1]))**2
        except:
            RMSE += (ground_truth[row[0].split("_landmark")[0]+".png"] - float(row[1]))**2
        num_images += 1.0
    fp2.close()
    RMSE = np.sqrt(RMSE/num_images)
print("RMSE=%.4f"%RMSE)

RMSE=0.5649


In [11]:
row[0]

'me1'

In [12]:
ground_truth

{'20171226_203858.jpg': 1.0,
 '20180727_203313.jpg': 1.0,
 '20190530_164944.jpg': 1.0,
 '20200108_084050.jpg': 3.0,
 'IMG_20170412_185554.jpg': 2.0,
 'jongun.jpg': 1.0,
 'me1.png': 1.0,
 'set1.jpg': 5.0,
 'set1_clean.jpg': 1.0,
 'set2.jpg': 3.0,
 'set2_clean.jpg': 1.0,
 'Soyeon Choi.jpg': 1.0,
 'soyeon.jpg': 1.0}